# California Housing Prediction Dashboard
This demo will consume a REST API fronting the latest trained model.

In [ ]:
from src.environment import init
import pandas as pd
import panel as pn
import warnings
import numpy as np

warnings.filterwarnings("ignore")
np.random.seed(42)

experiment_id, client = init()

In [ ]:
from src.constants import *

# Define our sliders
longitude_float_slider = pn.widgets.EditableFloatSlider(
    name="Longitude", start=-124, end=-115, step=0.1, value=DEFAULT_LONGITUDE
)
latitude_float_slider = pn.widgets.EditableFloatSlider(
    name="Latitude", start=32, end=42, step=0.1, value=DEFAULT_LATITUDE
)
housing_median_age_slider = pn.widgets.EditableFloatSlider(
    name="Housing Median Age", start=1, end=52, step=0.5, value=DEFAULT_MEDIAN_HOUSING_AGE
)
bedrooms_per_person_slider = pn.widgets.EditableFloatSlider(
    name="Bedrooms Per Person", start=0.1, end=1.0, step=0.01, value=DEFAULT_BEDRMS_PER_PERSON
)
population_slider = pn.widgets.EditableFloatSlider(
    name="Population", start=3, end=36000, step=360, value=DEFAULT_POPULATION
)
households_slider = pn.widgets.EditableFloatSlider(
    name="Households", start=2, end=6100, step=60, value=DEFAULT_HOUSEHOLDS
)
median_income_slider = pn.widgets.EditableFloatSlider(
    name="Median Income", start=0, end=15, step=0.15, value=DEFAULT_MEDIAN_INCOME
)

In [ ]:
def build_blank_result_row() -> pd.DataFrame:
    return pd.DataFrame(
        [
            {
                "longitude": [],
                "latitude": [],
                "housing_median_age": [],
                "bedrooms_per_person": [],
                "population": [],
                "households": [],
                "median_income": [],
                "median_housing_value": [],
            }
        ]
    )

In [ ]:
results_df_pane = pn.widgets.DataFrame(build_blank_result_row(), width=1300)

In [ ]:
def clear_btn_action(event):
    results_df_pane.value = build_blank_result_row()

In [ ]:
def reset_btn_action(event):
    longitude_float_slider.value = DEFAULT_LONGITUDE
    latitude_float_slider.value = DEFAULT_LATITUDE
    housing_median_age_slider.value = DEFAULT_MEDIAN_HOUSING_AGE
    bedrooms_per_person_slider.value = DEFAULT_BEDRMS_PER_PERSON
    population_slider.value = DEFAULT_POPULATION
    households_slider.value = DEFAULT_HOUSEHOLDS
    median_income_slider.value = DEFAULT_MEDIAN_INCOME

In [ ]:
from src.rest import invoke_rest_endpoint
from ae5_tools import demand_env_var


def submit_btn_action(event):
    feature_data: list[dict] = [
        {
            "longitude": longitude_float_slider.value,
            "latitude": latitude_float_slider.value,
            "housing_median_age": housing_median_age_slider.value,
            "population": population_slider.value,
            "households": households_slider.value,
            "median_income": median_income_slider.value,
            "diag_coord": longitude_float_slider.value + latitude_float_slider.value,
            "bedperroom": bedrooms_per_person_slider.value,
            "median_housing_value": [],
        }
    ]

    results: dict = invoke_rest_endpoint(
        endpoint_url=demand_env_var(name="SELF_HOSTED_MODEL_ENDPOINT"),
        input_data={"dataframe_records": feature_data},
        auth=False,
    )

    predicted_value: int = int(round(results["predictions"][0]))
    feature_data[0]["median_housing_value"] = predicted_value if predicted_value >= 0 else None

    row_df = pd.DataFrame(feature_data)

    if results_df_pane.value.loc[[0]]["median_housing_value"][0] == []:
        results_df_pane.value = row_df
    else:
        results_df_pane.value = pd.concat([results_df_pane.value, row_df], ignore_index=True)

In [ ]:
# Define our buttons
submit_btn = pn.widgets.Button(name="Submit", button_type="primary")
submit_btn.on_click(submit_btn_action)

clear_btn = pn.widgets.Button(name="Clear", button_type="primary")
clear_btn.on_click(clear_btn_action)

reset_btn = pn.widgets.Button(name="Reset Sliders", button_type="primary")
reset_btn.on_click(reset_btn_action)

In [ ]:
description_text: str = """
# Feature Desciptions

| Feature | Description                                                               |
| ------- | ------------------------------------------------------------------------- |
| longitude and latitude | Geospatial features for the region within California.      |
| housing_median_age     | The median age of housing. (years)                         |
| bedrooms_per_person    | The number of bedrooms per person.                         |
| households             | The number of houses in the region.                        |
| median_income          | The median income of the population. (thousands per month) |
"""
features_description_markdown = pn.pane.Markdown(object=description_text)

In [ ]:
pn.extension()

title: str = "<h1>Housing Prices Dashboard</h1>"
description = pn.pane.HTML("""Feature selection and predicted value""")

# Build layout
pn.Column(
    pn.Row(pn.Column(title, description)),
    pn.Row(
        pn.Column(
            longitude_float_slider,
            latitude_float_slider,
            housing_median_age_slider,
            bedrooms_per_person_slider,
            population_slider,
            households_slider,
            median_income_slider,
            reset_btn,
        ),
        pn.Column(pn.Row(submit_btn, clear_btn), results_df_pane),
    ),
    pn.Row(pn.Column(features_description_markdown)),
).servable("Housing Prices Dashboard")